<a href="https://colab.research.google.com/github/MickaRiv/ProjetDatascientest-VoiceSeparator/blob/main/streamlit/streamlit_voice-separation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prérequis



*   Avoir un compte ngrok sur https://dashboard.ngrok.com
*   récupérer le numéro de token



#Installations des paquets nécessaires

##pip install streamlit

In [1]:
!pip install streamlit

##pip install pygnrok

In [2]:
!pip install pyngrok

## pip install nussl

In [3]:
!pip install nussl

##pip install pydub

In [4]:
!pip install pydub

##pip install demucs

In [5]:
!pip install demucs

##pip install openunmix

In [6]:
!pip install openunmix

##pip install spleeter

In [7]:
!pip install spleeter

## pip install voicesep

In [8]:
!pip install git+https://github.com/MickaRiv/ProjetDatascientest-VoiceSeparator

  Cloning https://github.com/MickaRiv/ProjetDatascientest-VoiceSeparator to /tmp/pip-req-build-7ihu272u
  Running command git clone -q https://github.com/MickaRiv/ProjetDatascientest-VoiceSeparator /tmp/pip-req-build-7ihu272u


# Montage du drive

In [10]:
from google.colab import drive # import drive from google colab
drive.mount("/content/drive")  # we mount the google drive at /content/drive

Mounted at /content/drive


# Start de ngrok

In [10]:
!ls
!ngrok authtoken 25QRE4uYk7AG52BigElvRGk5TZc_4EfgeeiWXnGXYY2Qf5axB
!ngrok

drive  image.jpg  mix.wav  musdb18_temp  pred.wav  sample_data	tmp  voice.wav
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostna

#st_voice-sepration.py

##Chargement de l'image sur le git

In [15]:
!wget "https://raw.githubusercontent.com/MickaRiv/ProjetDatascientest-VoiceSeparator/main/streamlit/image.jpg"

--2022-03-08 13:27:37--  https://raw.githubusercontent.com/MickaRiv/ProjetDatascientest-VoiceSeparator/main/streamlit/image.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11459 (11K) [image/jpeg]
Saving to: ‘image.jpg’

image.jpg           100%[===================>]  11.19K  --.-KB/s    in 0s      

2022-03-08 13:27:38 (98.1 MB/s) - ‘image.jpg’ saved [11459/11459]



## récupération et exécution de st_voice-sepration.py du git

In [ ]:
!wget https://raw.githubusercontent.com/MickaRiv/ProjetDatascientest-VoiceSeparator/main/streamlit/st_voice-separation.py

--2022-02-25 14:04:07--  https://raw.githubusercontent.com/MickaRiv/ProjetDatascientest-VoiceSeparator/main/streamlit/st_voice-separation.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6143 (6.0K) [text/plain]
Saving to: ‘st_voice-separation.py’

st_voice-separation 100%[===================>]   6.00K  --.-KB/s    in 0s      

2022-02-25 14:04:07 (42.1 MB/s) - ‘st_voice-separation.py’ saved [6143/6143]



In [ ]:
!streamlit run --server.port 80 st_voice-separation.py &>/dev/null& # en background

## exécution du .py dans drive pour éviter les commit push à répétition

In [12]:
!ls /content/drive/MyDrive/'Projet Datascientest'/Ephi_st

st_voice-separation.py


In [12]:
!streamlit run --server.port 80 /content/drive/MyDrive/'Projet Datascientest'/Ephi_st/st_voice-separation.py &>/dev/null&

## ou dev direct dans colab

In [ ]:
%%writefile st_voice-separation_from_colab.py
import streamlit as st

st.title("st.title")


Overwriting st_voice-separation.py


In [ ]:
!streamlit run --server.port 80 st_voice-separation.py &>/dev/null& # en background

#Lancement du streamlit

In [13]:
from pyngrok import ngrok
# Setup u tunnel to the streamlit port 8501
public_url = ngrok.connect(port='80')
public_url

<NgrokTunnel: "http://0da0-34-125-147-9.ngrok.io" -> "http://localhost:80">

#TEST

In [1]:
import pandas as pd
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns
import nussl
import os
import numpy as np
import tensorflow as tf
from voicesep.core import get_musdb_data
from voicesep.unet import UNetModel
from pydub import AudioSegment
from os import listdir
from os.path import isfile, isdir, join
import shutil
from tensorflow.keras.models import load_model
from voicesep.demucs import DemucsModel
from voicesep.spleeter import SpleeterModel
from voicesep.openunmix import OpenUnmixModel

#-------------------------------------
# Définition des fonctions 
#-------------------------------------
@st.cache
# Récupération du listing des musiques 
def get_all_dataviz_csv():
    url = os.path.join('C:\\Users','magla','Documents','GitHub','ProjetDatascientest-VoiceSeparator','data','liste_musdb18_complet.csv')
    url = "https://raw.githubusercontent.com/MickaRiv/ProjetDatascientest-VoiceSeparator/main/data/liste_musdb18_complet.csv"
    return pd.read_csv(url)

def myloss(y_true, y_pred):
  return tf.math.reduce_sum(abs(y_true - y_pred)) + abs(tf.math.reduce_sum(y_true) - tf.math.reduce_sum(y_pred))

#musdb_path = os.path.join('C:\\Users','magla','Documents',"Projet_DataScientest","musdb18")
#unets_path = os.path.join('C:\\Users','magla','Documents',"Projet_DataScientest","UNet")

# Si colab et drive monté
musdb_path = os.path.join("/content","drive","MyDrive","Projet Datascientest","musdb18")
unets_path = os.path.join("/content","drive","MyDrive","Projet Datascientest","UNet")
#-------------------------------------
# Le streamlit 
#-------------------------------------
#%% dev de la sidebar avec chargement du modèle --------------------------

frequence = "Demucs"

   
if frequence=="Demucs" or frequence=="Spleeter" or frequence=="OpenUnmix":
    tmp_path = "tmp"
    if not os.path.exists(tmp_path):
        os.mkdir(tmp_path)
    in_path = os.path.join(tmp_path,"input")
    if not os.path.exists(in_path):
        os.mkdir(in_path)
    out_path = os.path.join(tmp_path,"output")
    if not os.path.exists(out_path):
        os.mkdir(out_path)
#%% ------------------------------------------
stem_ou_mp3 = 'stem'

if stem_ou_mp3 == 'stem':
    train_ou_test = 'test'
    musdb18path = os.path.join(musdb_path,train_ou_test)
    musdb18songs = [f for f in listdir(musdb18path) if isfile(join(musdb18path, f))]

    titre_musdb = "AM Contra - Heart Peripheral.stem.mp4"
    
    musdbsong = os.path.join(musdb18path,titre_musdb)
    musdb18 = 'musdb18_temp'
    if not os.path.exists(musdb18):
        os.mkdir(musdb18)

    shutil.copyfile(musdbsong, os.path.join(musdb18,titre_musdb))
    musdb_demo = get_musdb_data(gather_accompaniment=True,folder=os.getcwd(),subfolder=musdb18)
    
    data = musdb_demo[0]
    os.remove(os.path.join(musdb18,titre_musdb))
    
    signal = data["mix"]
    if frequence=="Demucs":
        unet_separator = DemucsModel(signal,"mdx_extra_q",in_path,out_path)
    elif frequence=="Spleeter":
        unet_separator = SpleeterModel(signal,in_path,out_path)
    elif frequence=="OpenUnmix":
        unet_separator = OpenUnmixModel(signal,"umx")

    audio_pred = unet_separator()[1]
    audio_pred.write_audio_to_file('pred.wav')
    
else:
    uploaded_file = st.file_uploader("Importez une musique",type=['wav','mp3'])

    if uploaded_file is not None:
    #st.write(uploaded_file.name)
        if uploaded_file.name.endswith('wav'):
            audio = AudioSegment.from_wav(uploaded_file)
            file_type = 'wav'
        elif uploaded_file.name.endswith('mp3'):
            audio = AudioSegment.from_mp3(uploaded_file)
            file_type = 'mp3'
            
        col1, col2 = st.columns(2)

        with col1:
            "**Le mix :**"
            audio.export(uploaded_file.name, format=file_type)
            st.audio(uploaded_file.name, format=file_type)
            signal = nussl.AudioSignal(uploaded_file.name)
            fig = plt.figure(figsize=(12, 6))
            nussl.utils.visualize_spectrogram(signal, y_axis='mel')
            st.pyplot(fig)
        
        with col2:
            "**La voix prédite :**"
            signal = nussl.AudioSignal(uploaded_file.name)
            
            if frequence=="8 kHz" or frequence=="4 kHz":
                unet_separator = UNetModel(signal,unet,freq,window_length,hop_length,patch_size,nfreq)
            elif frequence=="Demucs":
                unet_separator = DemucsModel(signal,"mdx_extra_q",in_path,out_path)
            elif frequence=="Spleeter":
                unet_separator = SpleeterModel(signal,in_path,out_path)
            elif frequence=="OpenUnmix":
                unet_separator = OpenUnmixModel(signal,"umx")
   
            audio_pred = unet_separator()[1]
            audio_pred.write_audio_to_file('pred.wav')
            fig = plt.figure(figsize=(12, 6))
            nussl.utils.visualize_spectrogram(audio_pred, y_axis='mel')


2022-03-08 14:30:22.523 INFO    root: Generating grammar tables from /usr/lib/python3.7/lib2to3/Grammar.txt
2022-03-08 14:30:22.569 INFO    root: Generating grammar tables from /usr/lib/python3.7/lib2to3/PatternGrammar.txt
/bin/sh: 1: sox: not found
2022-03-08 14:30:24.569 WARNING sox: SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    
2022-03-08 14:30:25.264474: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Going to separate the files:
tmp/input/mix.wav
With command:  python3 -m demucs.separate -o tmp/output -n mdx_extra_q


KeyboardInterrupt: ignored

# Arrêt de streamlit

Récupération de l'ID du process

In [14]:
!ps -eaf | grep streamlit

root        2291       1 22 14:40 ?        00:00:12 /usr/bin/python3 /usr/local/bin/streamlit run --server.port 80 /content/drive/MyDrive/Projet Datascientest/Ephi_st/st_voice-separation.py
root        2327    1889  0 14:41 ?        00:00:00 /bin/bash -c ps -eaf | grep streamlit
root        2329    2327  0 14:41 ?        00:00:00 grep streamlit


Kill du process

In [15]:
!kill 2291

Arrêt de ngrok

In [16]:
ngrok.kill()